In [403]:
from collections import namedtuple

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [137]:
class Task:
    Parameters = namedtuple('Parameters', 'O C D T A')

    def __init__(self, parameters) -> None:
        self.parameters = parameters

    def __str__(self) -> str:
        return str(self.parameters)


In [138]:
filename = 'data/example.txt'
tasks = []

with open(filename) as fp:
    n = fp.readline()
    n = int(n)

    for i in range(n):
        line = fp.readline()
        line = line.strip()
        params = [int(v) for v in line.split()]

        task = Task( Task.Parameters(*params) )
        tasks.append(task)

    

In [506]:
def compute_H(tasks):
    periods = [task.parameters.T for task in tasks]
    H = np.lcm.reduce(periods)
    return H

In [554]:
def get_ticks(tasks, for_deadlines=False, for_dummy=False):
    H = compute_H(tasks) if tasks else 0

    ticks = []
    ids = []
    if for_dummy:
        ticks = list( range(H) )
        ids = [0] * H
        tasks = []
    
    for i, task in enumerate(tasks):
        offset = task.parameters.O
        upper = H
        if for_deadlines:
            offset += task.parameters.D
            upper += task.parameters.D

        _ticks = list( range(offset, upper, task.parameters.T) )
        ticks += _ticks
        ids += [i + 1] * len(_ticks)

    df_ticks = pd.DataFrame( zip(ticks, ids), columns=['t', 'ids'])
    df_ticks['size'] = 1
    df_ticks['r'] = 60
    df_ticks['symbol'] = 'circle-cross-open'

    if for_deadlines:
        df_ticks['r'] = 50
        df_ticks['symbol'] = 'circle-x-open'
    elif for_dummy:
        df_ticks['size'] = 0

    df_ticks['ids'] = df_ticks['ids'].astype(str)

    # Format ticks
    df_ticks['t'] = df_ticks['t'] % H
    df_ticks['t'] = df_ticks['t'].astype(str)
    
    return df_ticks

In [582]:
def plot_events(df_ticks):
    fig = px.scatter_polar(df_ticks, r='r', theta='t', color='ids', size='size', 
                        symbol='symbol', symbol_map='identity',
                        start_angle=0, width=1000, height=1000,
                        direction='counterclockwise', color_discrete_sequence=px.colors.qualitative.Alphabet)
    return fig

In [586]:
df_ticks_releases = get_ticks(tasks)
df_ticks_deadlines = get_ticks(tasks, True)
df_ticks_dummy = get_ticks(tasks, for_dummy=True)

In [587]:
fig_releases = plot_events(df_ticks_releases)
fig_deadlines = plot_events(df_ticks_deadlines)

In [588]:
fig = plot_events(df_ticks_dummy)
for i in range(len(tasks)):
    fig.add_trace(fig_releases.data[i])
    fig.add_trace(fig_deadlines.data[i])

fig.show()

In [598]:
# px.pie?

In [592]:
H = compute_H(tasks)

In [594]:
values = [1] * H

df_trace = pd.DataFrame(values, columns=['value'])
# df_trace['theta'] = df_trace['theta'].astype(str)
# df_trace['r'] = 1
df_trace

,value
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [596]:
fig_trace = px.pie(df_trace, values='value', 
                                # symbol='symbol', symbol_map='identity',
                                # start_angle=0, width=1000, height=1000,
                                # direction='counterclockwise', color_discrete_sequence=px.colors.qualitative.Alphabet
                                )
fig_trace.show()